In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("IPL_Practise_New").getOrCreate()

24/12/21 16:16:58 WARN Utils: Your hostname, Ayushs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
24/12/21 16:16:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 16:16:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, BooleanType, DateType, DecimalType
from pyspark.sql import functions as f
from pyspark.sql import Window 

In [7]:
ball_by_ball_schema = StructType([
    StructField("match_id", IntegerType(), True),
    StructField("over_id", IntegerType(), True),
    StructField("ball_id", IntegerType(), True),
    StructField("innings_no", IntegerType(), True),
    StructField("team_batting", StringType(), True),
    StructField("team_bowling", StringType(), True),
    StructField("striker_batting_position", IntegerType(), True),
    StructField("extra_type", StringType(), True),
    StructField("runs_scored", IntegerType(), True),
    StructField("extra_runs", IntegerType(), True),
    StructField("wides", IntegerType(), True),
    StructField("legbyes", IntegerType(), True),
    StructField("byes", IntegerType(), True),
    StructField("noballs", IntegerType(), True),
    StructField("penalty", IntegerType(), True),
    StructField("bowler_extras", IntegerType(), True),
    StructField("out_type", StringType(), True),
    StructField("caught", BooleanType(), True),
    StructField("bowled", BooleanType(), True),
    StructField("run_out", BooleanType(), True),
    StructField("lbw", BooleanType(), True),
    StructField("retired_hurt", BooleanType(), True),
    StructField("stumped", BooleanType(), True),
    StructField("caught_and_bowled", BooleanType(), True),
    StructField("hit_wicket", BooleanType(), True),
    StructField("obstructingfeild", BooleanType(), True),
    StructField("bowler_wicket", BooleanType(), True),
    StructField("match_date", StringType(), True),
    StructField("season", IntegerType(), True),
    StructField("striker", IntegerType(), True),
    StructField("non_striker", IntegerType(), True),
    StructField("bowler", IntegerType(), True),
    StructField("player_out", IntegerType(), True),
    StructField("fielders", IntegerType(), True),
    StructField("striker_match_sk", IntegerType(), True),
    StructField("strikersk", IntegerType(), True),
    StructField("nonstriker_match_sk", IntegerType(), True),
    StructField("nonstriker_sk", IntegerType(), True),
    StructField("fielder_match_sk", IntegerType(), True),
    StructField("fielder_sk", IntegerType(), True),
    StructField("bowler_match_sk", IntegerType(), True),
    StructField("bowler_sk", IntegerType(), True),
    StructField("playerout_match_sk", IntegerType(), True),
    StructField("battingteam_sk", IntegerType(), True),
    StructField("bowlingteam_sk", IntegerType(), True),
    StructField("keeper_catch", BooleanType(), True),
    StructField("player_out_sk", IntegerType(), True),
    StructField("matchdatesk", StringType(), True),
])

In [8]:
ball_by_ball_df = spark.read.schema(ball_by_ball_schema).csv('files/Ball_By_Ball.csv', header=True)
ball_by_ball_df = ball_by_ball_df.withColumn("match_date",f.to_date(ball_by_ball_df['match_date'],"M/d/yyyy")).withColumn("matchdatesk",f.to_date(ball_by_ball_df['matchdatesk'],"yyyyMMdd"))
ball_by_ball_df.show()

24/12/21 16:17:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------+-------+----------+------------+------------+------------------------+----------+-----------+----------+-----+-------+----+-------+-------+-------------+--------------+------+------+-------+----+------------+-------+-----------------+----------+----------------+-------------+----------+------+-------+-----------+------+----------+--------+----------------+---------+-------------------+-------------+----------------+----------+---------------+---------+------------------+--------------+--------------+------------+-------------+-----------+
|match_id|over_id|ball_id|innings_no|team_batting|team_bowling|striker_batting_position|extra_type|runs_scored|extra_runs|wides|legbyes|byes|noballs|penalty|bowler_extras|      out_type|caught|bowled|run_out| lbw|retired_hurt|stumped|caught_and_bowled|hit_wicket|obstructingfeild|bowler_wicket|match_date|season|striker|non_striker|bowler|player_out|fielders|striker_match_sk|strikersk|nonstriker_match_sk|nonstriker_sk|fielder_match_sk|

In [9]:
match_schema = StructType([
       StructField("match_sk", IntegerType(), True),
       StructField("match_id", IntegerType(), True),
       StructField("team1", StringType(), True),
       StructField("team2", StringType(), True),
       StructField("match_date", StringType(), True),
       StructField("season_year", IntegerType(), True),
       StructField("venue_name", StringType(), True),
       StructField("city_name", StringType(), True),
       StructField("country_name", StringType(), True),
       StructField("toss_winner", StringType(), True),
       StructField("match_winner", StringType(), True),
       StructField("toss_name", StringType(), True),
       StructField("win_type", StringType(), True),
       StructField("outcome_type", StringType(), True),
       StructField("manofmach", StringType(), True),
       StructField("win_margin", IntegerType(), True),
       StructField("country_id", IntegerType(), True)
])

In [10]:
match_df = spark.read.schema(match_schema).csv('files/Match.csv',header=True)
match_df = match_df.withColumn("match_date",f.to_date(match_df['match_date'],"M/d/yyyy"))
match_df.show()

+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-------------+----------+----------+
|match_sk|match_id|               team1|               team2|match_date|season_year|          venue_name| city_name|country_name|         toss_winner|        match_winner|toss_name|win_type|outcome_type|    manofmach|win_margin|country_id|
+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-------------+----------+----------+
|       0|  335987|Royal Challengers...|Kolkata Knight Ri...|2008-04-18|       2008|M Chinnaswamy Sta...| Bangalore|       India|Royal Challengers...|Kolkata Knight Ri...|    field|    runs|      Result|  BB McCullum|       140|         1|
|       1|  335988|     Kings XI Punjab|

In [11]:
player_schema = StructType([
    StructField("player_sk", IntegerType(), True),
    StructField("player_id", IntegerType(), True),
    StructField("player_name", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("batting_hand", StringType(), True),
    StructField("bowling_skill", StringType(), True),
    StructField("country_name", StringType(), True)
])

In [12]:
player_df = spark.read.schema(player_schema).csv('files/Player.csv',header=True)
player_df = player_df.withColumn("dob",f.to_date(player_df['dob'],'M/d/yyyy'))
player_df.show()

+---------+---------+---------------+----------+--------------+--------------------+------------+
|player_sk|player_id|    player_name|       dob|  batting_hand|       bowling_skill|country_name|
+---------+---------+---------------+----------+--------------+--------------------+------------+
|        0|        1|     SC Ganguly|1972-07-08| Left-hand bat|    Right-arm medium|       India|
|        1|        2|    BB McCullum|1981-09-27|Right-hand bat|    Right-arm medium| New Zealand|
|        2|        3|     RT Ponting|1974-12-19|Right-hand bat|    Right-arm medium|   Australia|
|        3|        4|      DJ Hussey|1977-07-15|Right-hand bat|  Right-arm offbreak|   Australia|
|        4|        5|Mohammad Hafeez|1980-10-17|Right-hand bat|  Right-arm offbreak|    Pakistan|
|        5|        6|       R Dravid|1973-01-11|Right-hand bat|  Right-arm offbreak|       India|
|        6|        7|       W Jaffer|1978-02-16|Right-hand bat|  Right-arm offbreak|       India|
|        7|        8

In [13]:
player_match_schema = StructType([
    StructField("player_match_sk", IntegerType(), True),
    StructField("playermatch_key", DecimalType(), True),
    StructField("match_id", IntegerType(), True),
    StructField("player_id", IntegerType(), True),
    StructField("player_name", StringType(), True),
    StructField("dob", DateType(), True),
    StructField("batting_hand", StringType(), True),
    StructField("bowling_skill", StringType(), True),
    StructField("country_name", StringType(), True),
    StructField("role_desc", StringType(), True),
    StructField("player_team", StringType(), True),
    StructField("opposit_team", StringType(), True),
    StructField("season_year", IntegerType(), True),
    StructField("is_manofthematch", BooleanType(), True),
    StructField("age_as_on_match", IntegerType(), True),
    StructField("isplayers_team_won", BooleanType(), True),
    StructField("batting_status", StringType(), True),
    StructField("bowling_status", StringType(), True),
    StructField("player_captain", StringType(), True),
    StructField("opposit_captain", StringType(), True),
    StructField("player_keeper", StringType(), True),
    StructField("opposit_keeper", StringType(), True)
])

In [14]:
player_match_df = spark.read.schema(player_match_schema).csv('files/Player_match.csv',header=True)
player_match_df.show()

+---------------+---------------+--------+---------+---------------+----+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+------------------+--------------+--------------+--------------+---------------+-------------+--------------+
|player_match_sk|playermatch_key|match_id|player_id|    player_name| dob|  batting_hand|       bowling_skill|country_name|role_desc|         player_team|        opposit_team|season_year|is_manofthematch|age_as_on_match|isplayers_team_won|batting_status|bowling_status|player_captain|opposit_captain|player_keeper|opposit_keeper|
+---------------+---------------+--------+---------+---------------+----+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+------------------+--------------+--------------+--------------+---------------+-------------+--------------+
|            

In [15]:
team_schema = StructType([
    StructField("team_sk",IntegerType(),True),
    StructField("team_id",IntegerType(),True),
    StructField("team_name",StringType(),True)
])

In [16]:
team_df = spark.read.schema(team_schema).csv('files/Team.csv',header=True)

Transformation

In [17]:
ball_by_ball_df.show(5)

+--------+-------+-------+----------+------------+------------+------------------------+----------+-----------+----------+-----+-------+----+-------+-------+-------------+--------------+------+------+-------+----+------------+-------+-----------------+----------+----------------+-------------+----------+------+-------+-----------+------+----------+--------+----------------+---------+-------------------+-------------+----------------+----------+---------------+---------+------------------+--------------+--------------+------------+-------------+-----------+
|match_id|over_id|ball_id|innings_no|team_batting|team_bowling|striker_batting_position|extra_type|runs_scored|extra_runs|wides|legbyes|byes|noballs|penalty|bowler_extras|      out_type|caught|bowled|run_out| lbw|retired_hurt|stumped|caught_and_bowled|hit_wicket|obstructingfeild|bowler_wicket|match_date|season|striker|non_striker|bowler|player_out|fielders|striker_match_sk|strikersk|nonstriker_match_sk|nonstriker_sk|fielder_match_sk|

In [18]:
#Excluding Wide and NoBalls
ball_by_ball_df = ball_by_ball_df.filter((f.col("wides") == 0) & (f.col("noballs") == 0))

In [19]:
total_and_avg_runs = ball_by_ball_df.groupBy(["match_id","innings_no"]).agg(
    f.sum("runs_scored").alias("Total Runs Scored"),
    f.avg("runs_scored").alias("Avg Run Scored")
)

In [20]:
#Window Function
windowSpec = Window.partitionBy("match_id","innings_no").orderBy("over_id")

In [21]:
ball_by_ball_df = ball_by_ball_df.withColumn(
    "running_total_runs",
    f.sum("runs_scored").over(windowSpec)
)

In [22]:
ball_by_ball_df = ball_by_ball_df.withColumn(
    "high_impact_ball",f.when((f.col("runs_scored") + f.col("extra_runs") > 6) | (f.col("bowler_wicket") == True), True).otherwise(False))

In [23]:
match_df = match_df.withColumn("Year",f.year("match_date"))
match_df = match_df.withColumn("Month",f.month("match_date"))
match_df = match_df.withColumn("Day",f.dayofmonth("match_date"))

match_df = match_df.withColumn("win_mragin_category",
                               f.when(f.col('win_margin') >= 100, "High")
                               .when((f.col('win_margin') >= 50) & (f.col('win_margin') < 100), "Medium")
                               .otherwise("Low")
                               )

match_df = match_df.withColumn("toss_match_winner",
                               f.when(f.col("toss_winner") == f.col("match_winner"), "Yes")
                               .otherwise("No")
                               )

match_df.show()

+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-------------+----------+----------+----+-----+---+-------------------+-----------------+
|match_sk|match_id|               team1|               team2|match_date|season_year|          venue_name| city_name|country_name|         toss_winner|        match_winner|toss_name|win_type|outcome_type|    manofmach|win_margin|country_id|Year|Month|Day|win_mragin_category|toss_match_winner|
+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-------------+----------+----------+----+-----+---+-------------------+-----------------+
|       0|  335987|Royal Challengers...|Kolkata Knight Ri...|2008-04-18|       2008|M Chinnaswamy Sta...| Bangalore|     

In [24]:
player_df = player_df.na.fill({"batting_hand":"unknown", "bowling_skill":"unknown"})
player_df = player_df.withColumn("batting_style",f.when(f.col("batting_hand").contains("left"), "Left-Handed").otherwise("Right-Handed"))
player_df = player_df.withColumn("age",(f.datediff(f.current_date(), f.col('dob')) / 365.25).cast("int"))
player_df.show()

+---------+---------+---------------+----------+--------------+--------------------+------------+-------------+---+
|player_sk|player_id|    player_name|       dob|  batting_hand|       bowling_skill|country_name|batting_style|age|
+---------+---------+---------------+----------+--------------+--------------------+------------+-------------+---+
|        0|        1|     SC Ganguly|1972-07-08| Left-hand bat|    Right-arm medium|       India| Right-Handed| 52|
|        1|        2|    BB McCullum|1981-09-27|Right-hand bat|    Right-arm medium| New Zealand| Right-Handed| 43|
|        2|        3|     RT Ponting|1974-12-19|Right-hand bat|    Right-arm medium|   Australia| Right-Handed| 50|
|        3|        4|      DJ Hussey|1977-07-15|Right-hand bat|  Right-arm offbreak|   Australia| Right-Handed| 47|
|        4|        5|Mohammad Hafeez|1980-10-17|Right-hand bat|  Right-arm offbreak|    Pakistan| Right-Handed| 44|
|        5|        6|       R Dravid|1973-01-11|Right-hand bat|  Right-a

In [25]:
ball_by_ball_df.createOrReplaceTempView("ball_by_ball")
match_df.createOrReplaceTempView("match")
player_df.createOrReplaceTempView("player")
player_match_df.createOrReplaceTempView("player_match")
team_df.createOrReplaceTempView("team")

In [26]:
top_scoring_batsman = spark.sql("""
                                SELECT
                                P.PLAYER_NAME,M.SEASON_YEAR,SUM(B.RUNS_SCORED) AS TOTAL_RUNS
                                FROM BALL_BY_BALL B
                                INNER JOIN MATCH M ON B.MATCH_ID = M.MATCH_ID
                                INNER JOIN PLAYER_MATCH PM ON M.MATCH_ID = PM.MATCH_ID AND B.STRIKER = PM.PLAYER_ID
                                INNER JOIN PLAYER P ON P.PLAYER_ID = PM.PLAYER_ID
                                GROUP BY P.PLAYER_NAME,m.SEASON_YEAR
                                ORDER BY M.SEASON_YEAR, TOTAL_RUNS DESC
                                """)

In [27]:
top_scoring_batsman.show()

+-------------+-----------+----------+
|  PLAYER_NAME|SEASON_YEAR|TOTAL_RUNS|
+-------------+-----------+----------+
|     SE Marsh|       2008|       614|
|    G Gambhir|       2008|       532|
|ST Jayasuriya|       2008|       508|
|    SR Watson|       2008|       463|
|     GC Smith|       2008|       437|
| AC Gilchrist|       2008|       431|
|    YK Pathan|       2008|       430|
|     SK Raina|       2008|       420|
|     MS Dhoni|       2008|       414|
|    RG Sharma|       2008|       399|
|     V Sehwag|       2008|       399|
|     R Dravid|       2008|       370|
|   SC Ganguly|       2008|       349|
|     S Dhawan|       2008|       340|
|KC Sangakkara|       2008|       319|
|    DJ Hussey|       2008|       318|
|   RV Uthappa|       2008|       316|
|  SA Asnodkar|       2008|       311|
| Yuvraj Singh|       2008|       299|
|     PA Patel|       2008|       297|
+-------------+-----------+----------+
only showing top 20 rows

